In [1]:
import plotly.express as px
import pandas as pd
from lib.utils import load_thrasher_by_index,run_old_state_machine_on_thresholded_predictions,forward,run_new_state_machine_on_thresholded_predictions
from lib.models import MLP
import torch
from torch import nn
import numpy as np

/home/andrew/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df,(times,events),X = load_thrasher_by_index(1,dir='data/thrasher')
df

Length from shape: 3328.45
Length from timestamp: 3185.2010432730003
Length from epoch time : 3185.161999940872


,index,timestamp,acc_x,acc_y,acc_z,real time,activity,label,rawlabel,state,log
0,0.00,10885.336023,-0.725691,2.179468,9.678274,1.670366e+12,None,0.0,3.839151e-03,0,0.0
1,0.05,10885.383712,-0.881367,2.232158,9.692644,1.670366e+12,None,0.0,6.276207e-03,0,0.0
2,0.10,10885.431408,-1.262175,2.105222,9.393267,1.670366e+12,None,0.0,1.307537e-02,0,0.0
3,0.15,10885.479100,-1.005908,2.162703,9.649534,1.670366e+12,None,0.0,1.966746e-02,0,0.0
4,0.20,10885.526797,-0.929268,2.062112,9.604029,1.670366e+12,None,0.0,2.723153e-02,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
66564,3328.20,14070.345891,10.154883,-6.255791,-2.742297,1.670369e+12,None,0.0,1.244848e-10,0,0.0
66565,3328.25,14070.393679,10.964400,-5.407954,-1.717229,1.670369e+12,None,0.0,6.194638e-10,0,0.0
66566,3328.30,14070.441476,12.274476,-6.260581,-1.930386,1.670369e+12,None,0.0,6.243053e-10,0,0.0
66567,3328.35,14070.489271,11.445799,-7.300020,-2.258503,1.670369e+12,None,0.0,1.724553e-09,0,0.0


In [3]:
# let's visualize the data
fig = px.line(df,x='index',y=['acc_x','acc_y','acc_z'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [4]:
# let's visualize the raw network output
fig = px.line(df,x='index',y=['acc_x','acc_y','acc_z','rawlabel'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [5]:
# rawlabel output too small, let's increase magnitude
df['rawlabel'] = df['rawlabel']*10
fig = px.line(df,x='index',y=['acc_x','acc_y','acc_z','rawlabel'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [7]:
# how does this translate into puff detection, and therefore session detection?
# first we visualize thresholded rawlabels
df['label'] = df['label']*10
fig = px.line(df,x='index',y=['acc_x','acc_y','acc_z','rawlabel','label'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [6]:
# now we can get rid of rawlabels, understanding that the algorithm starts there. let's get rid of y and z also for now.
fig = px.line(df,x='index',y=['acc_x','label'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [8]:
# how do thresholded outputs translate into decisions about puffs?
fig = px.line(df,x='index',y=['acc_x','label','state'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [9]:
# so, where does the state machine decide there are puffs?
# get data to reset scaling by 10
df,(times,events),X = load_thrasher_by_index(1,dir='data/thrasher')
states,puff_locations = run_old_state_machine_on_thresholded_predictions(df['label'].to_numpy())
# df['state'] was computed on watch and we reimplemented here for consistency
states == df['state'].to_list()

Length from shape: 3328.45
Length from timestamp: 3185.2010432730003
Length from epoch time : 3185.161999940872


True

In [11]:
puff_locations

[91,
 531,
 5324,
 6885,
 7264,
 7807,
 8659,
 10669,
 10958,
 11612,
 12188,
 12350,
 12459,
 13212,
 13827,
 15649,
 24951,
 25810,
 27171,
 29763,
 30353,
 37562]

In [10]:
# let's plot puff locations
# how do thresholded outputs translate into decisions about puffs?
fig = px.line(df,y=['acc_x','label','state'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc)
fig.show(renderer='browser')

In [12]:
# let's, using the same acceleration signal, re-compute rawlabels
labels = forward(X)

100%|██████████| 66470/66470 [00:06<00:00, 10845.63it/s]


In [13]:
# let's compare rawlabels
df['rawlabel_new'] = labels
df['rawlabel'] = df['rawlabel']*10
df['rawlabel_new'] = df['rawlabel_new']*10
fig = px.line(df,x='index',y=['acc_x','rawlabel','rawlabel_new'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')

In [14]:
# how much total activation?
print(df['rawlabel'].sum())
print(df['rawlabel_new'].sum())

14079.588326581692
5640.556113118316


In [15]:
# do thresholding
label_new = [1 if y > .85 else 0 for y in labels]
df['label_new'] = label_new
df['label_new'] = df['label_new']*10
df['label'] = df['label']*10
fig = px.line(df,x='index',y=['acc_x','label','label_new'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
fig.show(renderer='browser')
# how much total activation?
print(df['label'].sum())
print(df['label_new'].sum())

8200.0
3660


In [16]:
states,puff_locations_new = run_old_state_machine_on_thresholded_predictions(label_new)
# df['state'] was computed on watch and we reimplemented here for consistency
states == df['state'].to_list()

False

In [17]:
puff_locations_new

[753, 1865, 4010, 5347, 7532, 8330, 30225, 37578]

In [18]:
df['state_new'] = states
print(df['state_new'].value_counts())
print(df['state'].value_counts())
# decrease in state 4 and state 2

0    64386
3     1382
4      435
1      252
2      114
Name: state_new, dtype: int64
0    62659
3     1755
4     1335
1      496
2      324
Name: state, dtype: int64


In [19]:
# let's plot puff locations
# how do thresholded outputs translate into decisions about puffs?
fig = px.line(df,y=['acc_x','label_new','state_new'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations_new:
    fig.add_vline(x=puff_loc)
fig.show(renderer='browser')

In [20]:
# compare old and new
fig = px.line(df,y=['acc_x','label','state','label_new','state_new'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc,line_dash="dash", line_color="red")
for puff_loc in puff_locations_new:
    fig.add_vline(x=puff_loc, line_color="green",line_width=2)
fig.show(renderer='browser')

In [ ]:
# which one's are sessions?
states,puff_locations_new_new = run_new_state_machine_on_thresholded_predictions(label_new)

In [ ]:
states == df['state_new'].to_list()

In [ ]:
df['state_new_new'] = states
# compare new and new new
fig = px.line(df,y=['acc_x','label_new','state_new','state_new_new'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations_new:
    fig.add_vline(x=puff_loc,line_dash="dash", line_color="red")
for puff_loc in puff_locations_new_new:
    fig.add_vline(x=puff_loc, line_color="green",line_width=2)
fig.show(renderer='browser')

In [ ]:
puff_locations_new

In [ ]:
# we gained 1 good-looking puff
puff_locations_new_new

In [77]:
# block to compare old puffs to new puffs
df,(times,events),X = load_thrasher_by_index(1,dir='data/thrasher')

states,puff_locations = run_old_state_machine_on_thresholded_predictions(df['label'].to_numpy())
labels = forward(X)
df['rawlabel_new'] = labels
df['rawlabel'] = df['rawlabel']*10
df['rawlabel_new'] = df['rawlabel_new']*10

label_new = [1 if y > .85 else 0 for y in labels]
df['label_new'] = label_new
df['label_new'] = df['label_new']*10
df['label'] = df['label']*10

states,puff_locations_new = run_new_state_machine_on_thresholded_predictions(label_new)
df['state_new'] = states

fig = px.line(df,y=['acc_x'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc,line_dash="dash", line_color="red")
for puff_loc in puff_locations_new:
    fig.add_vline(x=puff_loc, line_color="green",line_width=2)
fig.show(renderer='browser')

100%|██████████| 66470/66470 [00:06<00:00, 10808.71it/s]


In [21]:
# session detection
puffs = np.zeros(len(df))
puffs[np.array(puff_locations)] = 1
puffs_in_session = []
for i,puff in enumerate(puffs):
    if i < 9600:
        puffs_in_session.append(puffs[0:i+1].sum())
    else:
        puffs_in_session.append(puffs[i-9599:i+1].sum())
df['puff_count'] = puffs_in_session

In [22]:
session_idx = []
for i,(puff,puff_count) in enumerate(zip(puffs,puffs_in_session)):
    if(puff and puff_count > 2):
        session_idx.append(i)

In [23]:
# session detection
puffs = np.zeros(len(df))
puffs[np.array(puff_locations_new)] = 1
puffs_in_session = []
for i,puff in enumerate(puffs):
    if i < 9600:
        puffs_in_session.append(puffs[0:i+1].sum())
    else:
        puffs_in_session.append(puffs[i-9599:i+1].sum())
df['puff_count_new'] = puffs_in_session

In [24]:
session_idx_new = []
for i,(puff,puff_count) in enumerate(zip(puffs,puffs_in_session)):
    if(puff and puff_count > 2):
        session_idx_new.append(i)

In [26]:
fig = px.line(df[:50000],y=['acc_x','puff_count','puff_count_new'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc,line_dash="dash", line_color="red",line_width=.3)
for puff_loc in puff_locations_new:
    fig.add_vline(x=puff_loc, line_color="green",line_width=.3)
for session_loc in session_idx:
    fig.add_vline(x=session_loc,line_dash="dash", line_color="red",line_width=2)
for session_loc in session_idx_new:
    fig.add_vline(x=session_loc, line_color="green",line_width=2)
fig.show(renderer='browser')

In [27]:
# now the question is should a session have been triggered?
# we could compute using a label signal, but we don't have that.
fig = px.line(df,y=['acc_x'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations_new:
    fig.add_vline(x=puff_loc,line_dash='dash', line_color="green",line_width=1)
for session_loc in session_idx_new:
    fig.add_vline(x=session_loc, line_color="green",line_width=2)
fig.show(renderer='browser')

In [83]:
# block to compare old puffs to new puffs
df,(times,events),X = load_thrasher_by_index(1,dir='data/thrasher')
model = MLP().cuda()
model.load_state_dict(torch.load('model-epoch9.pt'))
logits = model(X.cuda())
y_pred = nn.Sigmoid()(logits)
y_pred = y_pred.detach().cpu()
y_pred = y_pred.round()
y_pred = torch.cat([torch.zeros(50,1),y_pred,torch.zeros(49,1)]).long()
df['y_pred'] = y_pred*10

labels = forward(X)
labels = [1 if y > .85 else 0 for y in labels]
states,puff_locations = run_new_state_machine_on_thresholded_predictions(labels)

df['label_1'] = labels
df['label_1'] = df['label_1']*10
df['state_1'] = states

states,puff_locations_new = run_new_state_machine_on_thresholded_predictions(y_pred)
df['label_2'] = y_pred
df['label_2'] = df['label_2']*10
df['state_2'] = states

fig = px.line(df,y=['acc_x','label_1','label_2'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc,line_dash="dash", line_color="red")
for puff_loc in puff_locations_new:
    fig.add_vline(x=puff_loc, line_color="green",line_width=2)
fig.show(renderer='browser')

Length from shape: 3328.45
Length from timestamp: 3185.2010432730003
Length from epoch time : 3185.161999940872


100%|██████████| 66470/66470 [00:06<00:00, 10518.25it/s]
